In [1]:
%load_ext autoreload
%autoreload 2
import datetime
import os
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from data_pipelines.data_preprocessing import DataPipeline
import pickle

# %load_ext blackcellmagic

In [2]:
cd ..

c:\Gan\GAN-market-prediction


In [3]:
base_path = Path(os.path.abspath('')) / 'data'
data_path = base_path / "stock_data"
sentiment_path = base_path /  'sentiment'
save_path = base_path / 'scaled_data' 

In [4]:
tickers = ["ATVI", "EA", "TTWO", "UBSFY"]
for ticker in tickers:
    pipeline = DataPipeline(
        ticker=ticker,
        data_path=data_path,
        save_path=save_path,
        sentiment_path=sentiment_path,
        if_ta=True,
        step_train=30,
        step_predict=1,
    )
    pipeline.save_data(sentiment=None)


In [ ]:
Wszedzie:
2 bez ta
3 sentiment
TTWO 
2 sentyment 
4 bez ta


In [5]:
data_path = Path(os.path.abspath('')) / 'data'
sentiment_path = data_path /  'sentiment' / 'UBSFY.csv'
stock_path = data_path /  'UBSFY.csv'

In [6]:
ea = pd.read_csv(sentiment_path).drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
ea = ea[ea["created_utc"] != "created_utc"]
ea["Date"] = (
    ea["created_utc"]
    .astype(int)
    .apply(lambda x: datetime.datetime.fromtimestamp(x).strftime("%Y-%m-%d"))
)

In [7]:
min_time = ea['Date'].min()
max_time = ea['Date'].max()

In [8]:
print(max_time)

2021-12-09


In [9]:
print(min_time)

2019-01-01


In [10]:
# stock_data = pd.read_csv(stock_path)

In [11]:
stock_data['Close'].min(), stock_data['Close'].max()

NameError: name 'stock_data' is not defined

In [ ]:
stock_data

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,sma_5,...,wma_50,bb_50_up,bb_50_down,sma_100,ema_100,wma_100,bb_100_up,bb_100_down,macd,signal
0,0,2018-12-11,47.637469,47.921902,46.803791,47.274574,12474126,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.274574,NaN,NaN,NaN,0.000000,0.000000
1,1,2018-12-12,48.108254,49.255792,47.490349,47.519775,11453591,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.398401,NaN,NaN,NaN,0.019560,0.003912
2,2,2018-12-13,47.814015,48.314221,46.804772,47.578621,7755190,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.459680,NaN,NaN,NaN,0.039356,0.011001
3,3,2018-12-14,46.558591,47.519775,46.068191,46.833214,7161356,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.298334,NaN,NaN,NaN,-0.005045,0.007792
4,4,2018-12-17,46.686096,47.490351,45.744528,46.176083,10064952,0.0,0,47.076453,...,NaN,NaN,NaN,NaN,47.064817,NaN,NaN,NaN,-0.092195,-0.012206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,752,2021-12-06,57.529999,58.450001,57.240002,58.090000,9408963,0.0,0,57.724000,...,67.369043,86.515702,55.688299,76.6782,75.181358,72.416444,93.540384,59.816016,-4.139811,-3.948483
753,753,2021-12-07,58.919998,59.270000,58.282001,58.770000,11553517,0.0,0,57.758000,...,66.885435,86.532406,55.019194,76.3479,74.856381,72.061826,93.306999,59.388801,-3.954544,-3.949695
754,754,2021-12-08,58.779999,60.009998,58.029999,59.270000,9785536,0.0,0,58.156001,...,66.434228,86.437224,54.431976,76.0376,74.547740,71.723650,93.100452,58.974748,-3.724440,-3.904644
755,755,2021-12-09,58.595001,59.810001,58.520000,59.070000,8402308,0.0,0,58.512000,...,65.988557,86.259199,53.877601,75.7132,74.241250,71.387658,92.820961,58.605439,-3.517670,-3.827249


In [ ]:
def rename(newname):
    def decorator(f):
        f.__name__ = newname
        return f
    return decorator

def q_at(y):
    @rename(f'q{y:0.2f}')
    def q(x):
        return x.quantile(y)
    return q

AGGREGATIONS = {
    'sentiment': ['count', 'mean', 'std', 'median', q_at(0.25), q_at(0.75)]
}

def prepare_sentiment_statistics(sentiment_data): 
    df = sentiment_data.copy()
    df['sentiment'] = df['sentiment'].astype(float)
    return df.groupby('Date').agg(AGGREGATIONS)['sentiment']

In [ ]:
ea_ready = prepare_sentiment_statistics(ea)

In [ ]:
for i in ea_ready.itertuples():
    print(i._asdict())

{'Index': '2019-01-01', 'count': 927, 'mean': 0.2659730784389428, 'std': 0.41558626672974086, 'median': 0.008676588, '_5': 0.00254704, '_6': 0.5362001}
{'Index': '2019-01-02', 'count': 18, 'mean': 0.14357090529444444, 'std': 0.2998072062708309, 'median': 0.036182085, '_5': 0.001855876175, '_6': 0.05887971875}
{'Index': '2019-01-03', 'count': 68, 'mean': 0.0890408249625, 'std': 0.2435692676867029, 'median': 0.030828463, '_5': 0.0009469751625000001, '_6': 0.037990276}
{'Index': '2019-01-04', 'count': 3, 'mean': 0.3255638676, 'std': 0.5520192129594781, 'median': 0.0068554264, '_5': 0.0068554264, '_6': 0.48491808820000004}
{'Index': '2019-01-05', 'count': 21, 'mean': 0.2775594083619048, 'std': 0.4230433876844926, 'median': 0.0060460465, '_5': 0.0016989658, '_6': 0.52522373}
{'Index': '2019-01-06', 'count': 45, 'mean': 0.35572840491222224, 'std': 0.41918257817139903, 'median': 0.13138996, '_5': 0.008486937, '_6': 0.89716285}
{'Index': '2019-01-07', 'count': 42, 'mean': 0.23051333987214284, 

In [ ]:
stock_data.set_index(pd.DatetimeIndex(stock_data['Date']))

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,sma_5,...,wma_50,bb_50_up,bb_50_down,sma_100,ema_100,wma_100,bb_100_up,bb_100_down,macd,signal
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-11,0,2018-12-11,47.637469,47.921902,46.803791,47.274574,12474126,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.274574,NaN,NaN,NaN,0.000000,0.000000
2018-12-12,1,2018-12-12,48.108254,49.255792,47.490349,47.519775,11453591,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.398401,NaN,NaN,NaN,0.019560,0.003912
2018-12-13,2,2018-12-13,47.814015,48.314221,46.804772,47.578621,7755190,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.459680,NaN,NaN,NaN,0.039356,0.011001
2018-12-14,3,2018-12-14,46.558591,47.519775,46.068191,46.833214,7161356,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.298334,NaN,NaN,NaN,-0.005045,0.007792
2018-12-17,4,2018-12-17,46.686096,47.490351,45.744528,46.176083,10064952,0.0,0,47.076453,...,NaN,NaN,NaN,NaN,47.064817,NaN,NaN,NaN,-0.092195,-0.012206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-06,752,2021-12-06,57.529999,58.450001,57.240002,58.090000,9408963,0.0,0,57.724000,...,67.369043,86.515702,55.688299,76.6782,75.181358,72.416444,93.540384,59.816016,-4.139811,-3.948483
2021-12-07,753,2021-12-07,58.919998,59.270000,58.282001,58.770000,11553517,0.0,0,57.758000,...,66.885435,86.532406,55.019194,76.3479,74.856381,72.061826,93.306999,59.388801,-3.954544,-3.949695
2021-12-08,754,2021-12-08,58.779999,60.009998,58.029999,59.270000,9785536,0.0,0,58.156001,...,66.434228,86.437224,54.431976,76.0376,74.547740,71.723650,93.100452,58.974748,-3.724440,-3.904644


In [ ]:
def prepare_data(X, X_cols, save_scaler_path, y_col="Close"):
    X_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    X_df = pd.DataFrame(X_scaler.fit_transform(X.loc[:, X_cols]), columns=X_cols)
    y_series = y_scaler.fit_transform(X.loc[:, y_col].values.reshape(-1, 1))
    X_df[y_col] = y_series
    scalers = {
        'X_scaler': X_scaler, 
        'y_scaler': y_scaler
    }
    with open(save_scaler_path / 'scalers.pickle', 'wb') as handle:
        pickle.dump(scalers, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return X_df

def split_data(X, step_train, step_predict, y_col="Close"):
    data_len = X.shape[0]
    X_list = []
    Y_preds_real_list = []
    Y_whole_real_list = []
    for i in range(data_len):
        X_step = X.loc[i : i + step_train - 1, [col for col in X.columns if col != y_col]]
        Y_pred_real = X.loc[i + step_train : i + step_train + step_predict - 1, y_col]
        Y_whole_real = X.loc[i : i + step_train - 1, y_col]
        if (len(X_step) == step_train) & (len(Y_pred_real) == step_predict):
            X_list.append(X_step)
            Y_preds_real_list.append(Y_pred_real)
            Y_whole_real_list.append(Y_whole_real)
    return np.array(X_list), np.array(Y_preds_real_list), np.array(Y_whole_real_list)

def train_test_split(data, train_percent):
    split_idx = round(len(data) * train_percent)
    return data[:split_idx], data[split_idx:]

In [ ]:
STEP_TRAIN = 30
STEP_PREDICT = 1

In [ ]:
save_path = Path(os.path.abspath('')) / 'data' / 'scaled_data' 

In [ ]:
def prepare_final_data(sentiment=None):
    final_data = (
        stock_data.loc[(stock_data["Date"] >= min_time) & (stock_data["Date"] < max_time)]
        .drop(columns=["Dividends", "Stock Splits", "Unnamed: 0"])
        .dropna(axis="columns")
    )
    print(final_data.columns)
    if sentiment is not None: 
        sentiment_df = prepare_sentiment_statistics(sentiment)
        final_data = (final_data.merge(sentiment_df, how='left', on='Date')
                                .set_index(pd.DatetimeIndex(final_data['Date']))
                                .drop(columns='Date')
                                .interpolate(method='time'))
    x_cols = [col for col in final_data.columns if col not in ["Close", "Date"]]
    scaled_data = prepare_data(X=final_data, X_cols=x_cols, save_scaler_path=save_path)
    X_list, Y_preds_real_list, Y_whole_real_list = split_data(
        scaled_data, STEP_TRAIN, STEP_PREDICT
    )
    return X_list, Y_preds_real_list, Y_whole_real_list

In [ ]:
X_list, Y_preds_real_list, Y_whole_real_list = prepare_final_data(None)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'sma_5', 'ema_5',
       'wma_5', 'bb_5_up', 'bb_5_down', 'sma_10', 'ema_10', 'wma_10',
       'bb_10_up', 'bb_10_down', 'ema_20', 'ema_50', 'ema_100', 'macd',
       'signal'],
      dtype='object')


In [ ]:
X_list[0].shape

(30, 19)

In [ ]:
Y_preds_real_list[0].shape

(1,)

In [ ]:
Y_whole_real_list[0].shape

(30,)

In [ ]:
df_lists = [X_list, Y_preds_real_list, Y_whole_real_list]
names = ['X_list', 'Y_preds_real_list', 'Y_whole_real_list']
temp = dict(zip(names, df_lists))
save_data = {}
for name, df_list in temp.items(): 
    train, test = train_test_split(df_list, 0.75)
    save_data[f'{name}_train'] = train
    save_data[f'{name}_test'] = test

with open(save_path / 'data.pickle', 'wb') as handle:
    pickle.dump(save_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
 test = (
        stock_data.loc[(stock_data["Date"] >= min_time) & (stock_data["Date"] < max_time)]
        .drop(columns=["Dividends", "Stock Splits", "Unnamed: 0"])
        .dropna(axis="columns")
    )

In [ ]:
len(X_list)

713